<a href="https://colab.research.google.com/github/SerhiiLoboda/Useful/blob/master/Yahoo_Finance_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import requests
from bs4 import BeautifulSoup
import csv 
import pandas as pd 
import datetime

In [0]:
#def for getting data from earnings page, including link to company page.In default today earning but accepts list of dates 
def earningsgeter (dates=datetime.date.today()):
  earnings=pd.DataFrame({'Date':[],"Symbol": [], "Link": [], "Companie": [], "Earnings_Call_Time": [], "EPS_Estimate": [],'EPS_Estimate': [] ,"Reported_EPS": [],"Surprise(%)": []})
  for date in dates:
    Symbols=[]
    Links=[]
    Companies=[]
    Earnings_Call_Times =[]
    EPS_Estimates   = []
    Reported_EPSs   =[]
    Surprises=[]
 
    EarningsUrl = "https://finance.yahoo.com/calendar/earnings?day="+str(date)
    r= requests.get(EarningsUrl)
    data=r.text
    soup=BeautifulSoup(data)
    for listing in soup.find_all('tr'):
      for Symbol in listing.find_all('td', attrs={'aria-label':"Symbol" }):
        Symbols.append(Symbol.text)
        for a in Symbol.find_all('a', href=True):
          Links.append('https://finance.yahoo.com'+a['href'])
      for Company in listing.find_all('td', attrs={'aria-label':"Company" }):
        Companies.append(Company.text)
      for Earnings_Call_Time in listing.find_all('td', attrs={'aria-label':"Earnings Call Time" }):
        Earnings_Call_Times.append(Earnings_Call_Time.text)
      for EPS_Estimate in listing.find_all('td', attrs={'aria-label':"EPS Estimate" }):
        EPS_Estimates.append(EPS_Estimate.text)
      for Reported_EPS in listing.find_all('td', attrs={'aria-label':"Reported EPS" }):
        Reported_EPSs.append(Reported_EPS.text)
      for Surprise in listing.find_all('td', attrs={'aria-label':"Surprise(%)" }):
        Surprises.append(Surprise.text)    
    earning=pd.DataFrame({"Symbol": Symbols, "Link": Links, "Companie": Companies, "Earnings_Call_Time": Earnings_Call_Times, "EPS_Estimate": EPS_Estimates,'EPS_Estimate': EPS_Estimates ,"Reported_EPS": Reported_EPSs,"Surprise(%)": Surprises})
    earning['Date']=date
    earnings=earnings.append(earning)
  return(earnings)

In [0]:
def newssgeter (CompanieLink,Companie, curentnewslinks=[]):
  r= requests.get(CompanieLink)
  data=r.text
  soup=BeautifulSoup(data)
  newLinks=[]
  newstimes=[]
  newsHeaders=[]
  newsTexts=[]
  for listing in soup.find_all('h3'):
    for a in listing.find_all('a', href=True):
      if not a['href'].lstrip().startswith('https') :
        r2= requests.get('https://finance.yahoo.com'+a['href'])
        newsLink='https://finance.yahoo.com'+a['href']
      else:
        r2= requests.get(a['href'])
        newsLink=a['href']           
      data2=r2.text
      soup2=BeautifulSoup(data2)
      try:
        newstime = datetime.datetime.strptime(soup2.find('time').attrs['datetime'], '%Y-%m-%dT%H:%M:%S.%fZ' )
        newLinks.append(newsLink)
        newstimes.append(newstime)
        newsHeaders.append(soup2.find('title').text)
        newsText=""
        for newsPart in soup2.find_all('p',attrs={'class':"canvas-atom", 'type':"text"  }):
          newsText=newsText +'\n '+newsPart.text
        for table in soup.find_all('table'):
          newsText=newsText +' <table> '
          for tr in table.find_all('tr'):
            newsText=newsText +'<tr> '
            for td in tr.find_all('td'):
              newsText=newsText +'<td> '+tr.text+' </td>'
            newsText=newsText +' </tr>'
          newsText=newsText +' </table>'
          newsText=newsText +'\n '+table.text
        newsTexts.append(newsText)
      except:
        continue     
  news=pd.DataFrame({"news time": newstimes, "news header": newsHeaders, "news text": newsTexts,"news link": newLinks })
  news['Companie']=Companie
  return(news) 

In [0]:
# Set dates for for financial reports period
start_date = datetime.datetime.strptime('2019-01-01', '%Y-%m-%d')
end_date = datetime.datetime.strptime('2019-01-05', '%Y-%m-%d')
earnings = earningsgeter([datetime.date.fromordinal(i) for i in range(start_date.toordinal(), end_date.toordinal())])

In [0]:
#Get list of companies, create dataframe for collected news
complist = earnings[['Link','Companie']].drop_duplicates().reset_index(drop=True)
aggNews=pd.DataFrame({"news time": [], "news header": [], "news text": [],"news link": [],"Companie": []})
#comppart=complist.iloc[:]
for line in complist.itertuples(name='Pandas'):   #comppart.itertuples(name='Pandas'):  
  news= newssgeter (line[1], line[2],aggNews['news link'].loc[aggNews["Companie"]==line[2]])
  aggNews=aggNews.append(news)